INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [1]:
# Library imports
import numpy as np

Create classes for modularity

In [2]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [3]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [4]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


<!-- Star -->

In [5]:
# Start of Optimizers

class Optimizer_SGD:
    # Initialize the optimizer with optional learning rate decay, momentum, and adaptive gradient
    def __init__(self, learning_rate=0.1, decay=0.001, momentum=0.9, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay  # Learning rate decay
        self.iterations = 0
        self.momentum = momentum  # Momentum parameter
        self.epsilon = epsilon  # Small value to prevent division by zero in AdaGrad

    # Call once before any parameter updates
    def pre_update_params(self):
        # If learning rate decay is set, apply it
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update the parameters
    def update_params(self, layer):
        # If using momentum
        if self.momentum:
            # If layer does not contain momentum arrays, create them filled with zeros
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # Build weight updates with momentum - take previous updates multiplied by retain factor
            # and update with current gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            
            # Build bias updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
        
        # Vanilla SGD updates (without momentum)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases
        
        # Update weights and biases
        layer.weights += weight_updates
        layer.biases += bias_updates
    
    def post_update_params(self):
        self.iterations += 1


class Optimizer_AdaGrad:
    # Initialize the optimizer with learning rate and epsilon
    def __init__(self, learning_rate=1.0, decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
    
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
    
    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays, create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        
        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        
        # Vanilla SGD parameter update + normalization with square rooted cache
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)
    
    def post_update_params(self):
        self.iterations += 1

Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [6]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [7]:
# Iris Dataset
# From the scikit-learn library
# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data # Features
# y = iris.target # Target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [8]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer
optimizer = Optimizer_SGD()

PERFORM ONLY 1 PASS

In [9]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss_function.forward(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

# Print the accuracy
print('acc:', accuracy)

acc: 0.2833333333333333


In [10]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

(300, 3)


In [11]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[-1.35764407e-05 -1.16799681e-04 -1.18804144e-04]
 [-1.84284257e-04  2.12485208e-04 -3.67557645e-06]]
[[0.00032457 0.00044709 0.00022888]]
[[-1.15501651e-04  1.13583775e-04  1.91787512e-06]
 [ 8.46726987e-05  4.44119217e-06 -8.91138909e-05]
 [ 7.88805762e-05 -2.59789839e-05 -5.29015923e-05]]
[[ 5.20228633e-06 -1.92710150e-06 -3.27518484e-06]]


TRAINING LOOP - 1000 EPOCHS

In [12]:
# Training loop for 1000 epochs
for epoch in range(1000):
    
    # FORWARD PASS
    # First layer
    dense1.forward(X)
    activation1.forward(dense1.output)
    
    # Second layer
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    
    # Calculate loss
    loss = loss_function.calculate(activation2.output, y)
    
    # Calculate accuracy
    predictions = np.argmax(activation2.output, axis=1)
    if len(y.shape) == 2:
        y_labels = np.argmax(y, axis=1)
    else:
        y_labels = y
    accuracy = np.mean(predictions == y_labels)
    
    # Print progress every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, Learning Rate: {optimizer.current_learning_rate}')
    
    # BACKWARD PASS (Backpropagation)
    # Backward through loss and softmax activation
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    
    # Backward through second dense layer
    dense2.backward(activation2.dinputs)
    
    # Backward through first activation and dense layer
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # UPDATE WEIGHTS AND BIASES
    # Apply learning rate decay before updating parameters
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

# Print final results
print(f'\nFinal Results:')
print(f'Epoch: {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

Epoch: 0, Loss: 1.0986, Accuracy: 0.2567, Learning Rate: 0.1
Epoch: 100, Loss: 1.0986, Accuracy: 0.3867, Learning Rate: 0.09099181073703368
Epoch: 200, Loss: 1.0973, Accuracy: 0.4033, Learning Rate: 0.08340283569641367
Epoch: 300, Loss: 1.0836, Accuracy: 0.4100, Learning Rate: 0.07698229407236336
Epoch: 400, Loss: 1.0782, Accuracy: 0.4267, Learning Rate: 0.07147962830593281
Epoch: 500, Loss: 1.0768, Accuracy: 0.4067, Learning Rate: 0.066711140760507
Epoch: 600, Loss: 1.0768, Accuracy: 0.4067, Learning Rate: 0.06253908692933084
Epoch: 700, Loss: 1.0768, Accuracy: 0.4033, Learning Rate: 0.05885815185403179
Epoch: 800, Loss: 1.0768, Accuracy: 0.4033, Learning Rate: 0.05558643690939411
Epoch: 900, Loss: 1.0768, Accuracy: 0.4033, Learning Rate: 0.0526592943654555

Final Results:
Epoch: 999, Loss: 1.0768, Accuracy: 0.4033


## OPTIMIZER COMPARISON

Compare different optimizers in terms of:
- How many epochs to stabilize the loss
- Final accuracy of the model

In [13]:
# Function to train and track metrics
def train_network(optimizer, epochs=1000, print_every=100):
    """
    Train the neural network and return training metrics
    """
    # Reinitialize the network layers to ensure fair comparison
    dense1_new = Layer_Dense(2, 3)
    activation1_new = Activation_ReLU()
    dense2_new = Layer_Dense(3, 3)
    activation2_new = Activation_Softmax()
    loss_function_new = Loss_CategoricalCrossEntropy()
    
    # Track metrics
    loss_history = []
    accuracy_history = []
    
    # Training loop
    for epoch in range(epochs):
        # FORWARD PASS
        dense1_new.forward(X)
        activation1_new.forward(dense1_new.output)
        dense2_new.forward(activation1_new.output)
        activation2_new.forward(dense2_new.output)
        
        # Calculate loss and accuracy
        loss = loss_function_new.calculate(activation2_new.output, y)
        predictions = np.argmax(activation2_new.output, axis=1)
        if len(y.shape) == 2:
            y_labels = np.argmax(y, axis=1)
        else:
            y_labels = y
        accuracy = np.mean(predictions == y_labels)
        
        # Store metrics
        loss_history.append(loss)
        accuracy_history.append(accuracy)
        
        # BACKWARD PASS
        loss_function_new.backward(activation2_new.output, y)
        activation2_new.backward(loss_function_new.dinputs)
        dense2_new.backward(activation2_new.dinputs)
        activation1_new.backward(dense2_new.dinputs)
        dense1_new.backward(activation1_new.dinputs)
        
        # UPDATE WEIGHTS
        optimizer.pre_update_params()
        optimizer.update_params(dense1_new)
        optimizer.update_params(dense2_new)
        optimizer.post_update_params()
    
    return loss_history, accuracy_history

In [14]:
# Function to find when loss stabilizes
def find_stabilization_epoch(loss_history, threshold=0.01, window=50):
    """
    Find the epoch where loss stabilizes (changes less than threshold for a window of epochs)
    """
    for i in range(window, len(loss_history)):
        recent_losses = loss_history[i-window:i]
        loss_change = max(recent_losses) - min(recent_losses)
        if loss_change < threshold:
            return i - window
    return len(loss_history) - 1  # If never stabilizes, return last epoch

In [15]:
print("=" * 70)
print("COMPARING OPTIMIZERS")
print("=" * 70)
print()

# Test 1: SGD with Momentum
print("Training with SGD + Momentum...")
optimizer1 = Optimizer_SGD(learning_rate=1.0, momentum=0.9, decay=0.01)
loss_history1, accuracy_history1 = train_network(optimizer1, epochs=1000)
stabilization1 = find_stabilization_epoch(loss_history1)
final_accuracy1 = accuracy_history1[-1]
print(f"✓ Training complete!")
print()

# Test 2: AdaGrad
print("Training with AdaGrad...")
optimizer2 = Optimizer_AdaGrad(learning_rate=1.0, decay=0.01)
loss_history2, accuracy_history2 = train_network(optimizer2, epochs=1000)
stabilization2 = find_stabilization_epoch(loss_history2)
final_accuracy2 = accuracy_history2[-1]
print(f"✓ Training complete!")
print()

# Display Results
print("=" * 70)
print("COMPARISON RESULTS")
print("=" * 70)
print()
print(f"{'Optimizer':<30} {'Stabilization Epoch':<25} {'Final Accuracy':<15}")
print("-" * 70)
print(f"{'SGD + Momentum':<30} {stabilization1:<25} {final_accuracy1:.4f}")
print(f"{'AdaGrad':<30} {stabilization2:<25} {final_accuracy2:.4f}")
print()
print("=" * 70)
print("ANALYSIS")
print("=" * 70)

# Determine winner
if stabilization1 < stabilization2:
    faster_optimizer = "SGD + Momentum"
    diff_epochs = stabilization2 - stabilization1
else:
    faster_optimizer = "AdaGrad"
    diff_epochs = stabilization1 - stabilization2

if final_accuracy1 > final_accuracy2:
    better_accuracy = "SGD + Momentum"
    acc_diff = (final_accuracy1 - final_accuracy2) * 100
else:
    better_accuracy = "AdaGrad"
    acc_diff = (final_accuracy2 - final_accuracy1) * 100

print(f"• Convergence Speed: {faster_optimizer} stabilized {diff_epochs} epochs faster")
print(f"• Accuracy: {better_accuracy} achieved {acc_diff:.2f}% higher accuracy")
print()

# Additional insights
print("Key Observations:")
print(f"  - Initial loss (epoch 0): SGD={loss_history1[0]:.4f}, AdaGrad={loss_history2[0]:.4f}")
print(f"  - Final loss (epoch 999): SGD={loss_history1[-1]:.4f}, AdaGrad={loss_history2[-1]:.4f}")
print(f"  - Loss reduction: SGD={((loss_history1[0] - loss_history1[-1])/loss_history1[0]*100):.2f}%, AdaGrad={((loss_history2[0] - loss_history2[-1])/loss_history2[0]*100):.2f}%")
print("=" * 70)

COMPARING OPTIMIZERS

Training with SGD + Momentum...
✓ Training complete!

Training with AdaGrad...
✓ Training complete!

COMPARISON RESULTS

Optimizer                      Stabilization Epoch       Final Accuracy 
----------------------------------------------------------------------
SGD + Momentum                 0                         0.3833
AdaGrad                        24                        0.3933

ANALYSIS
• Convergence Speed: SGD + Momentum stabilized 24 epochs faster
• Accuracy: AdaGrad achieved 1.00% higher accuracy

Key Observations:
  - Initial loss (epoch 0): SGD=1.0986, AdaGrad=1.0986
  - Final loss (epoch 999): SGD=1.0859, AdaGrad=1.0493
  - Loss reduction: SGD=1.15%, AdaGrad=4.49%
